In [67]:
import pandas as pd
import numpy as np
import requests
import bs4

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import time
import re
import os

In [2]:
#蒐集「全部年份的資料」
#第一隊
all_player_data  = []
web_url          = 'http://www.espn.com/nba/history/awards/_/id/44'
nba_top_team     = requests.get(web_url)
nba_top_team_bs4 = bs4.BeautifulSoup(nba_top_team.text,"lxml")
Table            = nba_top_team_bs4.find('table', class_ = 'tablehead')
players          = Table.find_all('tr',class_ = re.compile(r'\D{3,4}row'))
for player in players:
    player_data = []
    datas = player.find_all('td')
    for data in datas:
        player_data.append(data.text)
    all_player_data.append(player_data)

#將等於''的年份資料補齊
all_player_df = pd.DataFrame(all_player_data)
all_player_df[0] = all_player_df[0].fillna('bfill')


for row in range(len(all_player_df)):
    if all_player_df.loc[row,0] == '':
        all_player_df.loc[row,0] = all_player_df.loc[row-1,0]

#將年份資料變成int，順便過濾掉超過2014年的，因為官方數據只有到2014，保留需要的欄位，包括年份（對照該年的球員表現）姓名，位置，球隊（之後看勝率）
all_player_df[0]            = all_player_df[0].astype('int')
all_player_df_2014          = all_player_df[all_player_df[0]>=1984]
all_player_df_2014_需要的資料 = all_player_df_2014.iloc[:,:4]
all_player_df_2014_需要的資料.columns = ['Year','Player','Pos','Team']
all_player_1st              = all_player_df_2014_需要的資料

In [3]:
#蒐集「全部年份的資料」
#第二隊
all_player_data  = []
web_url          = 'http://www.espn.com/nba/history/awards/_/id/45'
nba_top_team     = requests.get(web_url)
nba_top_team_bs4 = bs4.BeautifulSoup(nba_top_team.text,"lxml")
Table            = nba_top_team_bs4.find('table', class_ = 'tablehead')
players          = Table.find_all('tr',class_ = re.compile(r'\D{3,4}row'))
for player in players:
    player_data = []
    datas = player.find_all('td')
    for data in datas:
        player_data.append(data.text)
    all_player_data.append(player_data)

#將等於''的年份資料補齊
all_player_df = pd.DataFrame(all_player_data)
all_player_df[0] = all_player_df[0].fillna('bfill')


for row in range(len(all_player_df)):
    if all_player_df.loc[row,0] == '':
        all_player_df.loc[row,0] = all_player_df.loc[row-1,0]

#將年份資料變成int，順便過濾掉超過2014年的，因為官方數據只有到2014，保留需要的欄位，包括年份（對照該年的球員表現）姓名，位置，球隊（之後看勝率）
all_player_df[0]            = all_player_df[0].astype('int')
all_player_df_2014          = all_player_df[all_player_df[0]>=1984]
all_player_df_2014_需要的資料 = all_player_df_2014.iloc[:,:4]
all_player_df_2014_需要的資料.columns = ['Year','Player','Pos','Team']
all_player_2nd              = all_player_df_2014_需要的資料

In [4]:
#蒐集「全部年份的資料」
#第三隊
all_player_data  = []
web_url          = 'http://www.espn.com/nba/history/awards/_/id/46'
nba_top_team     = requests.get(web_url)
nba_top_team_bs4 = bs4.BeautifulSoup(nba_top_team.text,"lxml")
Table            = nba_top_team_bs4.find('table', class_ = 'tablehead')
players          = Table.find_all('tr',class_ = re.compile(r'\D{3,4}row'))
for player in players:
    player_data = []
    datas = player.find_all('td')
    for data in datas:
        player_data.append(data.text)
    all_player_data.append(player_data)

#將等於''的年份資料補齊
all_player_df = pd.DataFrame(all_player_data)
all_player_df[0] = all_player_df[0].fillna('bfill')


for row in range(len(all_player_df)):
    if all_player_df.loc[row,0] == '':
        all_player_df.loc[row,0] = all_player_df.loc[row-1,0]

#將年份資料變成int，順便過濾掉超過2014年的，因為官方數據只有到2014，保留需要的欄位，包括年份（對照該年的球員表現）姓名，位置，球隊（之後看勝率）
all_player_df[0]            = all_player_df[0].astype('int')
all_player_df_2014          = all_player_df[all_player_df[0]>=1984]
all_player_df_2014_需要的資料 = all_player_df_2014.iloc[:,:4]
all_player_df_2014_需要的資料.columns = ['Year','Player','Pos','Team']
all_player_3rd              = all_player_df_2014_需要的資料

In [8]:
all_time_team = pd.DataFrame({})
all_time_team = all_time_team.append(all_player_1st)
all_time_team = all_time_team.append(all_player_2nd)
all_time_team = all_time_team.append(all_player_3rd)

In [81]:
all_time_team.to_csv('年度前三隊資料/歷年資料.csv',encoding='utf-8-sig',index=False,header=True)

In [24]:
all_dict = {}
all_time_team = all_time_team.sort_values('Year',ascending=True)
all_time_team['次數'] = 1
player_list           = all_time_team['Player'].unique()
for p in player_list:
    單一球員資料 = pd.DataFrame(all_time_team[all_time_team['Player']==p])
    單一球員資料['累積次數'] = 單一球員資料['次數'].cumsum()
    單一球員資料_reset_index = 單一球員資料.set_index('Year',drop=True)['累積次數'].to_dict()
    all_dict[p] = 單一球員資料_reset_index

### 以下是歷年進攻資料

In [35]:
#蒐集所有需要的url
urls   = []
path   = '/Users/andyhsu/Desktop/NBA年度前三隊預測/chromedriver'
driver = webdriver.Chrome(path)
url    = 'https://www.nba.com/stats/players/traditional/?sort=PTS&dir=-1&Season=2021-22&SeasonType=Regular%20Season'
driver.get(url)
time.sleep(3)
賽季選項 = Select(driver.find_element_by_xpath('//select[@name="Season"]'))
for x,y in enumerate(range(len(賽季選項.options[:10])),start=1):
    賽季選項.select_by_index(y)
    urls.append(driver.current_url)
driver.close()

In [41]:
player_data_dict = {}
for url in urls:
    球員資料_datas_list = []
    ####變出我要的年份，透過正則表達式，2021-22 to be 2022
    year = re.search(r'\d+-\d+',urls[0])[0]
    year_new = re.sub(r'\d{2}-','',year)


    driver = webdriver.Chrome(path)
    driver.get(url)
    time.sleep(5)
    選單選項 = Select(driver.find_element_by_xpath('//select[@ng-options="p for p in vm.data.pages"]'))
    選單選項.select_by_index(0)
    大表    = driver.find_element_by_xpath('//div[@class="nba-stat-table__overflow"]')
    內容    = 大表.find_element_by_xpath('./table/tbody')
    球員資料 = 內容.find_elements_by_xpath('./tr')
    for each_player_data in 球員資料:
        個別球員_datas_list = []
        個別球員_datas = each_player_data.find_elements_by_xpath('./td')
        for 個別球員_data in 個別球員_datas:
            個別球員_datas_list.append(個別球員_data.text)
        球員資料_datas_list.append(個別球員_datas_list)

        
    player_data_dict[year_new] = pd.DataFrame(球員資料_datas_list)
    driver.quit()

<selenium.webdriver.remote.webelement.WebElement (session="03176e378eea079ceedfdaef23d35377", element="4882160c-8629-4d60-932d-d7551a25ec35")>
1
Joel Embiid
PHI
28
68
45
23
33.8
30.6
9.8
19.6
49.9
1.4
3.7
37.1
9.6
11.8
81.4
2.1
9.6
11.7
4.2
3.1
1.1
1.5
2.7
55.5
46
2
5.4
2
LeBron James
LAL
37
56
25
31
37.2
30.3
11.4
21.8
52.4
2.9
8.0
35.9
4.5
6.0
75.6
1.1
7.1
8.2
6.2
3.5
1.3
1.1
2.2
53.0
21
6
-2.1
3
Giannis Antetokounmpo
MIL
27
67
45
22
32.9
29.9
10.3
18.6
55.3
1.1
3.6
29.3
8.3
11.4
72.2
2.0
9.6
11.6
5.8
3.3
1.1
1.4
3.2
56.5
46
4
5.9
4
Kevin Durant
BKN
33
55
36
19
37.2
29.9
10.5
20.3
51.8
2.1
5.5
38.3
6.8
7.4
91.0
0.5
6.9
7.4
6.4
3.5
0.9
0.9
2.1
50.3
18
4
4.9
5
Luka Doncic
DAL
23
65
44
21
35.4
28.4
9.9
21.6
45.7
3.1
8.8
35.3
5.6
7.5
74.4
0.9
8.3
9.1
8.7
4.5
1.2
0.6
2.2
53.1
44
10
2.2
6
Trae Young
ATL
23
76
40
36
34.9
28.4
9.4
20.3
46.0
3.1
8.0
38.2
6.6
7.3
90.4
0.7
3.1
3.7
9.7
4.0
0.9
0.1
1.7
46.5
42
0
2.1
7
DeMar DeRozan
CHI
32
76
43
33
36.1
27.9
10.2
20.2
50.4
0.7
1.9
35.2
6.8
7.8
87.

# 前三隊資料的合併

In [ ]:
def find_cumcount(player,year):
    if player not in all_dict.keys():
        num = 0
    else:
        while year not in all_dict[player].keys():
            #print(year)
            #print('-')
            #print(list(all_dict[player].keys())[-1])
            year += 1
            if year in all_dict[player].keys():
                num = all_dict[player][year]-1
                break
            elif year > list(all_dict[player].keys())[-1]:
                個人得獎最後年份  = list(all_dict[player].keys())[-1]
                num = all_dict[player][個人得獎最後年份]
                break
        else:
            num = all_dict[player][year]
    return num

In [79]:
for file in os.listdir('/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料'):
    print('/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/'+file)
    temp_file = pd.read_csv('/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/'+file)
    #這邊很重要，因為我們看2013資料的時候，不會知道他們入選了沒
    temp_file['累積前三隊'] = temp_file.apply(lambda x: find_cumcount(x['Player'],x['Year']-1),axis=1)
    temp_file.to_csv('/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/'+file,encoding='utf-8-sig',index=False,header=True)

/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2013球員資料.csv
/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2019球員資料.csv
/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2018球員資料.csv
/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2014球員資料.csv
/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2022球員資料.csv
/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2015球員資料.csv
/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2021球員資料.csv
/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2017球員資料.csv
/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2016球員資料.csv
/Users/andyhsu/Desktop/NBA年度前三隊預測/球員資料/2020球員資料.csv
